In [64]:
import numpy as np
import pandas as pd

In [65]:
df = pd.read_csv('/content/diabetes.csv')

In [66]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [67]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [68]:
X = df.drop(['Outcome'],axis = 1)
y = df['Outcome']

In [69]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [70]:
X.shape

(768, 8)

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 1)

In [126]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Input,Dropout

In [73]:
model = Sequential()
model.add(Dense(32,activation = 'relu',input_dim = 8))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(optimizer = 'Adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [74]:
model.fit(X_train,y_train,batch_size = 32,epochs = 10,validation_data = (X_test,y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.5475 - loss: 0.6973 - val_accuracy: 0.6753 - val_loss: 0.6462
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6782 - loss: 0.6284 - val_accuracy: 0.7078 - val_loss: 0.6045
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7090 - loss: 0.5916 - val_accuracy: 0.7273 - val_loss: 0.5739
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7213 - loss: 0.5570 - val_accuracy: 0.7208 - val_loss: 0.5508
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7283 - loss: 0.5476 - val_accuracy: 0.7532 - val_loss: 0.5348
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7517 - loss: 0.5404 - val_accuracy: 0.7532 - val_loss: 0.5213
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7608 - loss: 0.5214 - val_accuracy: 0.7597 - val_loss: 0.5101
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7736 - loss: 0.4973 - val_accuracy: 0.7662 - val_loss

In [75]:
#keras Tuner helps to automate this

How to select approprite optimizer

In [76]:
!pip install -U keras-tuner

In [77]:
!pip install keras-tuner --upgrade

In [78]:
import keras_tuner as kt

In [79]:
def build_model(hp):
  model = Sequential();
  model.add(Input(shape=(X_train.shape[1],)))
  model.add(Dense(32,activation = 'relu'))
  model.add(Dense(1,activation = 'sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer = optimizer,loss = 'binary_crossentropy',metrics = ['accuracy'])
  return model

In [80]:
tuner = kt.RandomSearch(build_model,objective = 'val_accuracy',max_trials = 5)

Reloading Tuner from ./untitled_project/tuner0.json


In [81]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

In [82]:
print(tuner.get_best_hyperparameters()[0].values)

{'optimizer': 'rmsprop'}


In [83]:
print(tuner.get_best_models(num_models = 1)[0])

<Sequential name=sequential, built=True>


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [84]:
model = tuner.get_best_models(num_models = 1)[0]

In [85]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [86]:
model.fit(X_train,y_train,batch_size = 32,epochs = 100,initial_epoch = 11,validation_data = (X_test,y_test))

Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7642 - loss: 0.5179 - val_accuracy: 0.7727 - val_loss: 0.5162
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7365 - loss: 0.5228 - val_accuracy: 0.7792 - val_loss: 0.5028
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7429 - loss: 0.5113 - val_accuracy: 0.7922 - val_loss: 0.4938
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7780 - loss: 0.4828 - val_accuracy: 0.7922 - val_loss: 0.4863
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7546 - loss: 0.4830 - val_accuracy: 0.7857 - val_loss: 0.4815
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7655 - loss: 0.4827 - val_accuracy: 0.7857 - val_loss: 0.4785
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7582 - loss: 0.4846 - val_accuracy: 0.7727 - val_loss: 0.4760
Epoch 19/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7582 - loss: 0.4690 - val_accuracy

how to find number of neurons

In [100]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units', min_value=8, max_value=128)
  #name,lower,upper,step_size

  model.add(Dense(units= units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [101]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name = 'without step size')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [102]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test),verbose=1)

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 18s


In [103]:
print(tuner.get_best_hyperparameters()[0].values)

{'units': 106}


In [104]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [105]:
model.fit(X_train,y_train,batch_size = 32,epochs = 100,initial_epoch = 6,validation_data = (X_test,y_test) )

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.7766 - loss: 0.4728 - val_accuracy: 0.7857 - val_loss: 0.4790
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7665 - loss: 0.4710 - val_accuracy: 0.7922 - val_loss: 0.4743
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7819 - loss: 0.4631 - val_accuracy: 0.7922 - val_loss: 0.4727
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7851 - loss: 0.4503 - val_accuracy: 0.7857 - val_loss: 0.4688
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7695 - loss: 0.4668 - val_accuracy: 0.7727 - val_loss: 0.4670
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7778 - loss: 0.4602 - val_accuracy: 0.7792 - val_loss: 0.4695
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7611 - loss: 0.4744 - val_accuracy: 0.7792 - val_loss: 0.4707
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7801 - loss: 0.4578 - val_accuracy: 0.77

how to select number of layer

In [106]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(72,activation='relu',input_dim =8))
  for i in range(hp.Int('num_layers',min_value = 1,max_value = 10)):
    model.add(Dense(72,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [107]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name = 'num_layer')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [108]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test),verbose=1)

Trial 5 Complete [00h 00m 06s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 25s


In [109]:
print(tuner.get_best_hyperparameters()[0].values)

{'num_layers': 6}


In [110]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [112]:
model.fit(X_train,y_train,batch_size = 32,epochs = 100,initial_epoch = 6,validation_data = (X_test,y_test) )

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.7725 - loss: 0.4577 - val_accuracy: 0.7857 - val_loss: 0.4777
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7926 - loss: 0.4084 - val_accuracy: 0.7987 - val_loss: 0.4655
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8115 - loss: 0.4113 - val_accuracy: 0.8052 - val_loss: 0.4602
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8152 - loss: 0.4252 - val_accuracy: 0.7403 - val_loss: 0.5357
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7993 - loss: 0.4247 - val_accuracy: 0.8052 - val_loss: 0.5163
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7940 - loss: 0.4092 - val_accuracy: 0.7987 - val_loss: 0.4772
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8014 - loss: 0.3970 - val_accuracy: 0.7857 - val_loss: 0.4881
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8320 - loss: 0.3656 - val_accuracy: 0.

Find Everything

In [127]:
def build_model(hp):
  model = Sequential()
  counter = 0
  # model.add(Dense(units= units,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value = 1,max_value = 10)):
    if counter == 0:
      model.add(
          Dense
           (units= hp.Int('units'+str(i),min_value = 8,max_value = 128),
            activation=hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid']),
            input_dim = 8)
           )
      model.add(Dropout(hp.Choice('dropout'+str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
        model.add(
          Dense
           (units= hp.Int('units'+str(i),min_value = 8,max_value = 128),
            activation=hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid'])
            )
           )
        model.add(Dropout(hp.Choice('dropout'+str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter +=1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta']),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model


In [128]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        directory="mydir",
                        project_name="Final1")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [129]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test),verbose=1)

Trial 10 Complete [00h 00m 08s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 01m 10s


In [130]:
print(tuner.get_best_hyperparameters()[0].values)

{'num_layers': 5, 'units0': 98, 'activation0': 'tanh', 'dropout0': 0.1, 'optimizer': 'rmsprop', 'units1': 61, 'activation1': 'relu', 'dropout1': 0.4, 'units2': 92, 'activation2': 'tanh', 'dropout2': 0.4, 'units3': 9, 'activation3': 'relu', 'dropout3': 0.1, 'units4': 27, 'activation4': 'relu', 'dropout4': 0.5, 'units5': 82, 'activation5': 'sigmoid', 'dropout5': 0.6, 'units6': 66, 'activation6': 'tanh', 'dropout6': 0.3, 'units7': 88, 'activation7': 'sigmoid', 'dropout7': 0.5, 'units8': 81, 'activation8': 'tanh', 'dropout8': 0.1, 'units9': 65, 'activation9': 'tanh', 'dropout9': 0.4}


In [131]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [132]:
model.fit(X_train,y_train,epochs = 100,initial_epoch = 6,validation_data = (X_test,y_test) )

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.7233 - loss: 0.5191 - val_accuracy: 0.7987 - val_loss: 0.4694
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7240 - loss: 0.5258 - val_accuracy: 0.7857 - val_loss: 0.4785
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7194 - loss: 0.5112 - val_accuracy: 0.7857 - val_loss: 0.4794
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7292 - loss: 0.5350 - val_accuracy: 0.7922 - val_loss: 0.4771
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7844 - loss: 0.4745 - val_accuracy: 0.8117 - val_loss: 0.4660
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7432 - loss: 0.4764 - val_accuracy: 0.8052 - val_loss: 0.4674
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7551 - loss: 0.4815 - val_accuracy: 0.7857 - val_loss: 0.4746
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7599 - loss: 0.5228 - val_accuracy: 0.80